In [1]:
! pip install opencv-python numpy

In [2]:
import os
import numpy as np 
import cv2

In [3]:
skip =[]
def process_video(video_index):
    region1 = (0, 0, 134, 240)
    region2 = (93, 0, 227, 240)  
    region3 = (186, 0, 320, 240)

    path = r'C:\Users\Neuralyx\Documents\Frames'
    roi_lists = []

    vid_path = os.path.join(path, 'vid%d' % video_index)
    
    roi_list = []
    
    frame_files = sorted(os.listdir(vid_path))
    
    def mean_squared_error(image_x, image_y):
        assert image_x.shape == image_y.shape
        mse = np.mean((image_y - image_x) ** 2)
        return mse
    
    def ssim(X, Y, C1, C2):
        mu_X = np.mean(X)
        mu_Y = np.mean(Y)

        sigma_X = np.var(X)
        sigma_Y = np.var(Y)

        sigma_XY = np.cov(X, Y)[0][1]

        numerator = ((2 * mu_X * mu_Y) + C1) * ((2 * sigma_XY) + C2)
        denominator = ((mu_X**2 + mu_Y**2) + C1) * ((sigma_X + sigma_Y) + C2)

        return numerator / denominator


    
    for i in range(len(frame_files) - 1):
        frame_path_1 = os.path.join(vid_path, frame_files[i])
        frame_path_2 = os.path.join(vid_path, frame_files[i + 1])

        image_1 = cv2.imread(frame_path_1)
        image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
        
        image_2 = cv2.imread(frame_path_2)
        image_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2RGB)

        image_reg_1 = image_1[:, region1[0]:region1[2]]
        image_reg_2 = image_1[:, region2[0]:region2[2]]
        image_reg_3 = image_1[:, region3[0]:region3[2]]
    
        image_reg_4 = image_2[:, region1[0]:region1[2]]
        image_reg_5 = image_2[:, region2[0]:region2[2]]
        image_reg_6 = image_2[:, region3[0]:region3[2]]
        
        mse_region_1 = mean_squared_error(image_reg_1, image_reg_4)
        mse_region_2 = mean_squared_error(image_reg_2, image_reg_5)
        mse_region_3 = mean_squared_error(image_reg_3, image_reg_6)
    
        max_mse_value = max(mse_region_1, mse_region_2, mse_region_3)
    
        if max_mse_value == mse_region_1:
            action_temp = image_reg_1
        elif max_mse_value == mse_region_2:
            action_temp = image_reg_2
        else:
            action_temp = image_reg_3
    
        overl1 =  image_1[:, 93:134]
        overl2 =  image_1[:, 186:227]
    
        im = cv2.cvtColor(action_temp, cv2.COLOR_BGR2GRAY)
        im = cv2.resize(im, (240, 41))
        im = im.flatten()
        
        im1 = cv2.cvtColor(overl1, cv2.COLOR_BGR2GRAY)
        im1 = im1.flatten()
        corr1 = np.corrcoef(im, im1)
        
        im2 = cv2.cvtColor(overl2, cv2.COLOR_BGR2GRAY)
        im2 = im2.flatten()
        corr2 = np.corrcoef(im, im2)
        
        if corr1[0, 1] > corr2[0, 1]:
            roi_list.append(im1)
        else:
            roi_list.append(im2)

    roi_lists.append(roi_list)
    print('video{%d} roi done' % video_index)

    ssim_lists = []

    for roi_list in roi_lists:
        ssim_list = []
        for i in range(len(roi_list)-1):
            el1 = roi_list[i]
            el2 = roi_list[(i+1)]
            c1 = 2.55 ** 2
            c2 = 7.65 ** 2
            ssim_list.append(ssim(el1, el2, c1, c2))
        ssim_lists.append(ssim_list)
        print('video{%d} ssim done' % video_index)

    primary_key_list = []
    alternate_key_list = []
    

    for ssim_list in ssim_lists:
        primary_key = []
        alternate_key = []
        for i in range(len(ssim_list)-1):
            if (0.65 < ssim_list[i]) and (ssim_list[i] < 0.90):
                image_path = os.path.join(vid_path, r'frame%d.jpg' % i)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                primary_key.append(image)
            
            if (0.65 < ssim_list[i]) and (ssim_list[i] < 0.97):
                image_path = os.path.join(vid_path, r'frame%d.jpg' % i)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                alternate_key.append(image)
        
#         primary_key_list.append(primary_key)
#         alternate_key_list.append(alternate_key)
        print('video{%d} keylist done' % video_index)
#         print(len(primary_key))
#         print(len(alternate_key))
    K = []
    nkf = 10

    if len(primary_key) >= nkf: 
        k_ratio = int(len(primary_key)/nkf)
        
        if(k_ratio != 0):
            primary_key_klist = primary_key[::k_ratio] 
            K.append(primary_key_klist)
        else:
            skip.append(video_index)
        
    else: 
        k_ratio = int(len(alternate_key)/nkf)
        
        if(k_ratio != 0):
            alternate_key_klist = alternate_key[::k_ratio] 
            K.append(alternate_key_klist)
        else:
            skip.append(video_index)
            
    print('video{%d} klist done' % video_index)

#     for video, frames in enumerate(K, start=1):
#         keyframe_path = r'D:\RESEARCH\keyframes\vid{}'.format(video)
#         os.makedirs(keyframe_path, exist_ok=True)
#         for count, image in enumerate(frames):
#             output_path = os.path.join(keyframe_path, 'frame{}.jpg'.format(count))
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             cv2.imwrite(output_path, image)

#     keyframe_path = r'D:\RESEARCH\keyframes\vid{%d}' % video_index
#     os.makedirs(keyframe_path, exist_ok=True)
#     for count, image in enumerate(K):
#         output_path = os.path.join(keyframe_path, 'frame{%d}.jpg' % count)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         cv2.imwrite(output_path, image)
#     print('video{%d} keyframe done' % video_index)

    keyframe_path = r'D:\RESEARCH\KeyFrames\vid%d' % video_index
    os.makedirs(keyframe_path, exist_ok=True)
    
    x=0
    
    for i, image_list in enumerate(K):
        for j, image in enumerate(image_list):
            output_path = os.path.join(keyframe_path, 'frame%d.jpg' % x)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            cv2.imwrite(output_path, image)
            x+=1



# Now, you can call this function for each video index
for video_index in range(30, 7010):
    process_video(video_index)
    print('video{%d} done' % video_index)
    


video{30} roi done
video{30} ssim done
video{30} keylist done
video{30} klist done
video{30} done
video{31} roi done
video{31} ssim done
video{31} keylist done
video{31} klist done
video{31} done
video{32} roi done
video{32} ssim done
video{32} keylist done
video{32} klist done
video{32} done
video{33} roi done
video{33} ssim done
video{33} keylist done
video{33} klist done
video{33} done
video{34} roi done
video{34} ssim done
video{34} keylist done
video{34} klist done
video{34} done
video{35} roi done
video{35} ssim done
video{35} keylist done
video{35} klist done
video{35} done
video{36} roi done
video{36} ssim done
video{36} keylist done
video{36} klist done
video{36} done
video{37} roi done
video{37} ssim done
video{37} keylist done
video{37} klist done
video{37} done
video{38} roi done
video{38} ssim done
video{38} keylist done
video{38} klist done
video{38} done
video{39} roi done
video{39} ssim done
video{39} keylist done
video{39} klist done
video{39} done
video{40} roi done
v

C:\Users\Neuralyx\anaconda3\envs\my_env\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]


video{41} roi done
video{41} ssim done
video{41} keylist done
video{41} klist done
video{41} done
video{42} roi done
video{42} ssim done
video{42} keylist done
video{42} klist done
video{42} done
video{43} roi done
video{43} ssim done
video{43} keylist done
video{43} klist done
video{43} done


C:\Users\Neuralyx\anaconda3\envs\my_env\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


video{44} roi done
video{44} ssim done
video{44} keylist done
video{44} klist done
video{44} done
video{45} roi done
video{45} ssim done
video{45} keylist done
video{45} klist done
video{45} done
video{46} roi done
video{46} ssim done
video{46} keylist done
video{46} klist done
video{46} done
video{47} roi done
video{47} ssim done
video{47} keylist done
video{47} klist done
video{47} done
video{48} roi done
video{48} ssim done
video{48} keylist done
video{48} klist done
video{48} done
video{49} roi done
video{49} ssim done
video{49} keylist done
video{49} klist done
video{49} done
video{50} roi done
video{50} ssim done
video{50} keylist done
video{50} klist done
video{50} done
video{51} roi done
video{51} ssim done
video{51} keylist done
video{51} klist done
video{51} done
video{52} roi done
video{52} ssim done
video{52} keylist done
video{52} klist done
video{52} done
video{53} roi done
video{53} ssim done
video{53} keylist done
video{53} klist done
video{53} done
video{54} roi done
v

...

video{6630} roi done
video{6630} ssim done
video{6630} keylist done
video{6630} klist done
video{6630} done
video{6631} roi done
video{6631} ssim done
video{6631} keylist done
video{6631} klist done
video{6631} done
video{6632} roi done
video{6632} ssim done
video{6632} keylist done
video{6632} klist done
video{6632} done
video{6633} roi done
video{6633} ssim done
video{6633} keylist done
video{6633} klist done
video{6633} done
video{6634} roi done
video{6634} ssim done
video{6634} keylist done
video{6634} klist done
video{6634} done
video{6635} roi done
video{6635} ssim done
video{6635} keylist done
video{6635} klist done
video{6635} done
video{6636} roi done
video{6636} ssim done
video{6636} keylist done
video{6636} klist done
video{6636} done
video{6637} roi done
video{6637} ssim done
video{6637} keylist done
video{6637} klist done
video{6637} done
video{6638} roi done
video{6638} ssim done
video{6638} keylist done
video{6638} klist done
video{6638} done
video{6639} roi done
video{6

video{6706} roi done
video{6706} ssim done
video{6706} keylist done
video{6706} klist done
video{6706} done
video{6707} roi done
video{6707} ssim done
video{6707} keylist done
video{6707} klist done
video{6707} done
video{6708} roi done
video{6708} ssim done
video{6708} keylist done
video{6708} klist done
video{6708} done
video{6709} roi done
video{6709} ssim done
video{6709} keylist done
video{6709} klist done
video{6709} done
video{6710} roi done
video{6710} ssim done
video{6710} keylist done
video{6710} klist done
video{6710} done
video{6711} roi done
video{6711} ssim done
video{6711} keylist done
video{6711} klist done
video{6711} done
video{6712} roi done
video{6712} ssim done
video{6712} keylist done
video{6712} klist done
video{6712} done
video{6713} roi done
video{6713} ssim done
video{6713} keylist done
video{6713} klist done
video{6713} done
video{6714} roi done
video{6714} ssim done
video{6714} keylist done
video{6714} klist done
video{6714} done
video{6715} roi done
video{6

video{6782} roi done
video{6782} ssim done
video{6782} keylist done
video{6782} klist done
video{6782} done
video{6783} roi done
video{6783} ssim done
video{6783} keylist done
video{6783} klist done
video{6783} done
video{6784} roi done
video{6784} ssim done
video{6784} keylist done
video{6784} klist done
video{6784} done
video{6785} roi done
video{6785} ssim done
video{6785} keylist done
video{6785} klist done
video{6785} done
video{6786} roi done
video{6786} ssim done
video{6786} keylist done
video{6786} klist done
video{6786} done
video{6787} roi done
video{6787} ssim done
video{6787} keylist done
video{6787} klist done
video{6787} done
video{6788} roi done
video{6788} ssim done
video{6788} keylist done
video{6788} klist done
video{6788} done
video{6789} roi done
video{6789} ssim done
video{6789} keylist done
video{6789} klist done
video{6789} done
video{6790} roi done
video{6790} ssim done
video{6790} keylist done
video{6790} klist done
video{6790} done
video{6791} roi done
video{6

video{6858} roi done
video{6858} ssim done
video{6858} keylist done
video{6858} klist done
video{6858} done
video{6859} roi done
video{6859} ssim done
video{6859} keylist done
video{6859} klist done
video{6859} done
video{6860} roi done
video{6860} ssim done
video{6860} keylist done
video{6860} klist done
video{6860} done
video{6861} roi done
video{6861} ssim done
video{6861} keylist done
video{6861} klist done
video{6861} done
video{6862} roi done
video{6862} ssim done
video{6862} keylist done
video{6862} klist done
video{6862} done
video{6863} roi done
video{6863} ssim done
video{6863} keylist done
video{6863} klist done
video{6863} done
video{6864} roi done
video{6864} ssim done
video{6864} keylist done
video{6864} klist done
video{6864} done
video{6865} roi done
video{6865} ssim done
video{6865} keylist done
video{6865} klist done
video{6865} done
video{6866} roi done
video{6866} ssim done
video{6866} keylist done
video{6866} klist done
video{6866} done
video{6867} roi done
video{6

video{6934} roi done
video{6934} ssim done
video{6934} keylist done
video{6934} klist done
video{6934} done
video{6935} roi done
video{6935} ssim done
video{6935} keylist done
video{6935} klist done
video{6935} done
video{6936} roi done
video{6936} ssim done
video{6936} keylist done
video{6936} klist done
video{6936} done
video{6937} roi done
video{6937} ssim done
video{6937} keylist done
video{6937} klist done
video{6937} done
video{6938} roi done
video{6938} ssim done
video{6938} keylist done
video{6938} klist done
video{6938} done
video{6939} roi done
video{6939} ssim done
video{6939} keylist done
video{6939} klist done
video{6939} done
video{6940} roi done
video{6940} ssim done
video{6940} keylist done
video{6940} klist done
video{6940} done
video{6941} roi done
video{6941} ssim done
video{6941} keylist done
video{6941} klist done
video{6941} done
video{6942} roi done
video{6942} ssim done
video{6942} keylist done
video{6942} klist done
video{6942} done
video{6943} roi done
video{6

In [4]:
print(K)

NameError: name 'K' is not defined

In [5]:
print(skip)

[31, 79, 98, 102, 122, 132, 233, 250, 280, 344, 356, 388, 391, 403, 441, 448, 567, 580, 631, 649, 651, 667, 691, 712, 716, 752, 778, 785, 786, 811, 821, 824, 831, 860, 867, 878, 920, 921, 931, 941, 947, 1006, 1010, 1026, 1033, 1038, 1040, 1051, 1057, 1086, 1105, 1109, 1137, 1154, 1168, 1184, 1196, 1201, 1230, 1235, 1246, 1251, 1269, 1287, 1316, 1330, 1354, 1408, 1426, 1429, 1442, 1463, 1478, 1484, 1496, 1505, 1533, 1541, 1560, 1571, 1584, 1587, 1616, 1648, 1650, 1655, 1694, 1709, 1712, 1822, 1852, 1858, 1870, 1874, 1927, 1933, 1941, 1944, 1947, 1949, 1956, 1960, 1967, 1975, 1995, 1999, 2002, 2013, 2038, 2064, 2066, 2078, 2082, 2097, 2110, 2113, 2136, 2158, 2172, 2183, 2194, 2226, 2231, 2264, 2287, 2300, 2327, 2328, 2331, 2382, 2385, 2394, 2410, 2418, 2431, 2462, 2470, 2476, 2477, 2494, 2527, 2528, 2553, 2557, 2581, 2610, 2616, 2625, 2661, 2665, 2675, 2695, 2731, 2752, 2786, 2805, 2808, 2810, 2842, 2856, 2886, 2897, 2956, 2969, 2982, 2993, 2998, 3027, 3030, 3033, 3042, 3068, 3070, 3102,

In [6]:
print(len(skip))

390


In [ ]:
skip_path = r'D:\RESEARCH\skiplist'
